In [1]:
import tensorflow as tf

tf.__version__

'1.15.0'

In [2]:
## GPU 메모리 관리 ##
#1번 방법
config = tf.ConfigProto()
config.gpu_options.allow_growth = True #메모리 수요에 따라 할당

'''주의할 점은, 이 옵션은 메모리의 증식만 가능하다는 것. 연산이 끝나고 메모리가 필요없는 상황이라고 해서 할당된 메모리를 반납하지 않는다. 

Tensorflow 측에서는 그럴 경우 더 심한 메모리 파편화를 일으킬 수도 있다고 하니 판단은 독자의 몫.'''

'주의할 점은, 이 옵션은 메모리의 증식만 가능하다는 것. 연산이 끝나고 메모리가 필요없는 상황이라고 해서 할당된 메모리를 반납하지 않는다. \n\nTensorflow 측에서는 그럴 경우 더 심한 메모리 파편화를 일으킬 수도 있다고 하니 판단은 독자의 몫.'

In [3]:
## GPU 메모리 관리 ##
#2번 방법
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5 #메모리 할당 비율 설정
#config.gpu_options.allow_growth = True #메모리 수요에 따라 할당, 두개 같이 사용 가능(추천하지는 않음)

In [4]:
##9.2 계산그래프를 만들어 세션에서 실행하기 ##
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

sess = tf.Session(config=config)
sess.run(x.initializer) #변수초기화
sess.run(y.initializer) #변수초기화
result = sess.run(f) #그래프평가(실행)

print(result)

sess.close()

42


In [5]:
## with를 사용한 방법 ##
with tf.Session(config=config) as sess:
    #x.initializer.run() #tf.get_default_graph().run(x.initializer)와 동일
    #y.initializer.run() #tf.get_default_graph().run(y.initializer)와 동일
    init = tf.global_variables_initializer()
    init.run() #모든 변수 초기화
    result = f.eval() #tf.get_default_graph().run(f)와 동일
    print(result)

42


In [6]:
sess = tf.InteractiveSession() #자기자신을 기본세션으로 지정
init.run()
result = f.eval()
print(result)
sess.close() #반드시 세션을 종료시켜야 함

42


In [7]:
## 9.3 계산 그래프 관리 ##
x1 = tf.Variable(1) #노드를 만들면 자동으로 기본 계산 그래프레 추가됨
x1.graph is tf.get_default_graph()

True

In [8]:
graph = tf.Graph() #독립적인 그래프 만들기
with graph.as_default(): #임시적으로 기본그래프로 사용
    x2 = tf.Variable(2)

print(x2.graph is graph) #독립적인 그래프에 속함
print(x2.graph is tf.get_default_graph()) #기본그래프에 속하지 않음

True
False


In [9]:
tf.reset_default_graph() #기본그래프 초기화, 중복된 노드 제거

In [10]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x + 3

with tf.Session() as sess:
    y_val, z_val = sess.run([y, z]) #변수는 세션이 유지되는 동안 유효함
    print(y_val)
    print(z_val)

10
8


In [11]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing("data/")
m, n = housing.data.shape
# bias 추가
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

print('data shape :', housing.data.shape)
print('data_plus_bias shape :', housing_data_plus_bias.shape)

data shape : (20640, 8)
data_plus_bias shape : (20640, 9)


<img src="https://render.githubusercontent.com/render/math?math=%5Chat%7B%5Cmathbf%7BW%7D%7D%20%3D%20%5Cleft%28%5Cmathbf%7BX%7D%5E%7BT%7D%20%5Cmathbf%7BX%7D%20%5Cright%29%5E%7B-1%7D%20%5Cmathbf%7BX%7D%5E%7BT%7D%20%5Cmathbf%7BY%7D&mode=display"/>

In [12]:
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
Y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="Y")
XT = tf.transpose(X) #전치행렬로 만듬(행과 열을 바꿈)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), Y) #정규방정식

with tf.Session(config=config) as sess:
    theta_value = theta.eval()
    
theta_value

array([[-3.68962631e+01],
       [ 4.36777472e-01],
       [ 9.44449380e-03],
       [-1.07348785e-01],
       [ 6.44962370e-01],
       [-3.94082872e-06],
       [-3.78797273e-03],
       [-4.20847952e-01],
       [-4.34020907e-01]], dtype=float32)

In [13]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [15]:
## 9.6 경사하강법 구현 ##
## 9.6.1 직접 그래디언트 계산 ##
n_epochs = 10000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
Y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="Y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta") #난수를 담은 텐서를 생성하는 노드를 그래프에 생성
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - Y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients) #변수에 새로운 값을 할당하는 노드 생성

init = tf.global_variables_initializer()

with tf.Session(config=config) as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 1000 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()
    
best_theta

Epoch 0 MSE =  7.940561
Epoch 1000 MSE =  0.52697515
Epoch 2000 MSE =  0.5244226
Epoch 3000 MSE =  0.5243249
Epoch 4000 MSE =  0.52432114
Epoch 5000 MSE =  0.524321
Epoch 6000 MSE =  0.524321
Epoch 7000 MSE =  0.524321
Epoch 8000 MSE =  0.524321
Epoch 9000 MSE =  0.524321


array([[ 2.0685523 ],
       [ 0.8296086 ],
       [ 0.11875082],
       [-0.26550704],
       [ 0.3056793 ],
       [-0.00450327],
       [-0.03932588],
       [-0.89990246],
       [-0.87055564]], dtype=float32)

In [35]:
## 9.6.2 자동미분사용 ##
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
Y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="Y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta") #난수를 담은 텐서를 생성하는 노드를 그래프에 생성
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - Y
mse = tf.reduce_mean(tf.square(error), name="mse")
#gradients = 2/m * tf.matmul(tf.transpose(X), error)
gradients = tf.gradients(mse, [theta])[0]
training_op = tf.assign(theta, theta - learning_rate * gradients) #변수에 새로운 값을 할당하는 노드 생성

init = tf.global_variables_initializer()

with tf.Session(config=config) as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

best_theta

Epoch 0 MSE =  14.611462
Epoch 100 MSE =  0.9247965
Epoch 200 MSE =  0.6902329
Epoch 300 MSE =  0.643217
Epoch 400 MSE =  0.61133003
Epoch 500 MSE =  0.5881644
Epoch 600 MSE =  0.5712756
Epoch 700 MSE =  0.55894274
Epoch 800 MSE =  0.54992145
Epoch 900 MSE =  0.5433097


array([[ 2.0685523 ],
       [ 0.83239436],
       [ 0.16031384],
       [-0.19400582],
       [ 0.21279533],
       [ 0.01058048],
       [-0.04290124],
       [-0.5943006 ],
       [-0.56156534]], dtype=float32)

In [27]:
## 9.6.3 옵티마이저 사용 ##
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
Y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="Y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta") #난수를 담은 텐서를 생성하는 노드를 그래프에 생성
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - Y
mse = tf.reduce_mean(tf.square(error), name="mse")
#gradients = tf.gradients(mse, [theta])[0]
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
#training_op = tf.assign(theta, theta - learning_rate * gradients) #변수에 새로운 값을 할당하는 노드 생성
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session(config=config) as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()
    
best_theta

Epoch 0 MSE =  8.389791
Epoch 100 MSE =  0.7047312
Epoch 200 MSE =  0.559149
Epoch 300 MSE =  0.547571
Epoch 400 MSE =  0.54105914
Epoch 500 MSE =  0.5364064
Epoch 600 MSE =  0.5330499
Epoch 700 MSE =  0.53062695
Epoch 800 MSE =  0.5288777
Epoch 900 MSE =  0.5276145


array([[ 2.0685523e+00],
       [ 7.9364920e-01],
       [ 1.3111897e-01],
       [-1.6182078e-01],
       [ 2.0421983e-01],
       [ 4.6267582e-04],
       [-3.9605416e-02],
       [-8.4350580e-01],
       [-8.0824536e-01]], dtype=float32)